# Scan pattern testing

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import proplot as pplt

sys.path.append('..')
from tools import utils

sys.path.append('/Users/46h/Research/btf/btf-scripts/')
import scan_patterns as sp

pplt.rc['grid'] = False

In [ ]:
ndim = 3
n = 16
slit_dict = {
    'x1': {
        'center': 0.0,
        'distance': 1.0,
        'steps': n,
        'min': -np.inf,
        'max': +np.inf,
    },
    'x2': {
        'center': 0.0,
        'distance': 1.0,
        'steps': n,
        'min': -np.inf,
        'max': +np.inf,
    },
    'x3': {
        'center': 0.0,
        'distance': 0.0,
        'steps': 0,
        'min': -np.inf,
        'max': +np.inf,
    },
}
for attr, sign in zip(['min', 'max'], [-1.0, +1.0]):
    slit_dict['x1'][attr] = sign * 0.4
    pass
    
keys = list(slit_dict)
M = np.identity(ndim)
M[keys.index('x3'), keys.index('x1')] = 1.0

In [ ]:
import importlib
importlib.reload(sp)

# Run points generator
navg = 1
kws = dict(
    slit_dict=slit_dict, 
    M=M, 
    reprate=5.0,
    navg=navg, 
    boundary='ellipsoid',  # {None, 'ellipsoid'} 
    R=1.0,
    exclude_outside_box=True,
)
lgen = list(sp.gen(**kws))

# Reshape
if navg > 0:
    pts = np.zeros((len(lgen), ndim))
    for i in range(len(lgen)):
        pts[i, :] = lgen[i][0]
else:
    lgen = np.array(lgen)
    pts = np.zeros((2 * lgen.shape[0], ndim))
    for i in range(ndim):
        pts[:, i] = lgen[:, i, :2].ravel()
    
# Un-shear new (generated) points.
center = np.array([slit_dict[key]['center'] for key in keys])
Minv = np.linalg.inv(M)
pts_n = utils.apply(Minv, pts - center)

In [ ]:
_pts = pts

In [ ]:
from plotly import graph_objects as go

fig = go.Figure(data=go.Scatter3d(x=_pts[:, 0], y=_pts[:, 1], z=_pts[:, 2], mode='markers',
                                  marker=dict(color='black', size=1.0, opacity=0.)))
# fig.update_scenes(xaxis_visible=False, yaxis_visible=False, zaxis_visible=False)
fig.show()

In [ ]:
labels = ['x1', 'x2', 'x3']
fig, axes = pplt.subplots(ncols=3, nrows=3, figwidth=5, span=False)
for i in range(3):
    for j in range(3):
        x = _pts[:, j]
        y = _pts[:, i]
        ax = axes[i, j]
        ax.plot(x, y, lw=1, marker='.', ms=2, color='black')
    axes[i, 0].format(ylabel=labels[i])
    axes[-1, i].format(xlabel=labels[i])
axes.format(xlim=(-0.55, 0.55), ylim=(-0.55, 0.55))

In [ ]:
np.max(pts, axis=0)